In [ ]:
import sys

from SVERL_icml_2023.portfolio_DRL.data_function import *
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import math
from torch.utils.data import Dataset, DataLoader
from SVERL_icml_2023.portfolio_DRL.create_model import *
from sklearn.model_selection import TimeSeriesSplit
from SVERL_icml_2023.shapley import Shapley
import shap
sys.path.append("E:/XAI/RL_with_SHAP/pythonProject1/SVERL_icml_2023")
# from portfolio_DRL.create_model import StockPredictWrapper
import pandas as pd
import numpy as np
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.callbacks import EvalCallback
import pandas as pd
import numpy as np
import gymnasium as gym
from gym import spaces
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
import os

In [7]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# Load data explicitly
shap_metrics = pd.read_csv('shap_value_metrics_export.csv', parse_dates=['Start_Date', 'End_Date'])
factor_returns = pd.read_csv('aligned_factors.csv', parse_dates=['Date'], index_col='Date')
stock_returns = pd.read_csv('daily_returns.csv', parse_dates=['Date'], index_col='Date')

# Explicitly filter only 'Test' phase SHAP metrics
shap_metrics_test = shap_metrics[shap_metrics['Phase'] == 'Test'].copy()

# Verify explicitly
print(f"Total SHAP records (Test Phase): {len(shap_metrics_test)}")

# Select SHAP metrics explicitly
selected_columns = [
    'mean_abs_shap_Mkt-RF', 'mean_abs_shap_SMB', 'mean_abs_shap_HML',
    'mean_abs_shap_RMW', 'mean_abs_shap_CMA',
    'shap_std_Mkt-RF', 'shap_std_SMB', 'shap_std_HML',
    'shap_std_RMW', 'shap_std_CMA',
    'mean_abs_over_std_Mkt-RF', 'mean_abs_over_std_SMB', 'mean_abs_over_std_HML',
    'mean_abs_over_std_RMW', 'mean_abs_over_std_CMA'
]

# Aggregate SHAP metrics explicitly by Start_Date and End_Date across stocks
shap_agg = shap_metrics_test.groupby(['Start_Date', 'End_Date'])[selected_columns].mean().reset_index()

# Explicitly scale SHAP metrics
scaler_shap = StandardScaler()
shap_agg[selected_columns] = scaler_shap.fit_transform(shap_agg[selected_columns])

# Verify explicitly scaled SHAP metrics
print("Scaled SHAP metrics (Test phase):", shap_agg.head())

# Explicitly map SHAP intervals to daily returns
daily_regimes = pd.DataFrame(index=stock_returns.index)

for _, row in shap_agg.iterrows():
    mask = (daily_regimes.index >= row['Start_Date']) & (daily_regimes.index <= row['End_Date'])
    daily_regimes.loc[mask, selected_columns] = row[selected_columns].values

# Explicit forward-fill for continuity
daily_regimes.ffill(inplace=True)
daily_regimes.dropna(inplace=True)

# Verify explicitly aligned daily regimes
print("Daily regimes (aligned):", daily_regimes.head())

# Normalize explicitly factor returns
factor_scaler = StandardScaler()
factor_returns_norm = pd.DataFrame(
    factor_scaler.fit_transform(factor_returns),
    index=factor_returns.index,
    columns=factor_returns.columns
)

# Normalize explicitly stock returns
stock_scaler = StandardScaler()
stock_returns_norm = pd.DataFrame(
    stock_scaler.fit_transform(stock_returns),
    index=stock_returns.index,
    columns=stock_returns.columns
)

# Merge explicitly into final dataset
merged_data = stock_returns_norm.join(daily_regimes, how='inner').join(factor_returns_norm, rsuffix='_factor').dropna()

# Explicit final dataset verification
print("Merged normalized data preview:", merged_data.head())

# Save explicitly final aligned dataset
merged_data.to_csv("final_merged_data.csv")

start_date = merged_data.index.min()
end_date = merged_data.index.max()

# Explicitly filter raw stock returns to match the merged_data dates
raw_stock_returns = stock_returns.loc[start_date:end_date].copy()
raw_stock_returns.to_csv("aligned_raw_stock_returns.csv")

stock_returns_norm_filter = stock_returns_norm.loc[start_date:end_date].copy()

Total SHAP records (Test Phase): 12964
Scaled SHAP metrics (Test phase):   Start_Date   End_Date  mean_abs_shap_Mkt-RF  mean_abs_shap_SMB  \
0 2006-06-09 2006-06-22              0.746145           0.844451   
1 2006-06-23 2006-07-07              0.591442           1.321563   
2 2006-07-10 2006-07-21              0.701747           1.144933   
3 2006-07-24 2006-08-04              0.235944           0.440245   
4 2006-08-07 2006-08-18              0.154516           0.162799   

   mean_abs_shap_HML  mean_abs_shap_RMW  mean_abs_shap_CMA  shap_std_Mkt-RF  \
0          -0.052021           0.381862           0.096910         0.011057   
1           0.365858          -0.052443           0.296386         0.154940   
2           0.386793           0.880681           0.881446         0.060338   
3           0.654832          -0.166087           0.466175        -0.063344   
4           0.291578           0.199758           0.683642        -0.078770   

   shap_std_SMB  shap_std_HML  shap_std_RMW

In [8]:
import gym
from gym import spaces
import numpy as np

class SinglePPOEnv(gym.Env):
    def __init__(self, returns_df, regime_df, raw_returns_df):
        super().__init__()
        self.stock_returns = returns_df.values
        self.regime_features = regime_df.values
        self.raw_stock_returns = raw_returns_df.values
        self.current_step = 0
        self.n_assets = self.stock_returns.shape[1]

        self.action_space = spaces.Box(low=0, high=1, shape=(self.n_assets,), dtype=np.float32)

        # Adjust observation space shape accordingly
        obs_dim = self.n_assets + self.regime_features.shape[1]
        self.observation_space = spaces.Box(
            low=-np.inf, high=np.inf,
            shape=(obs_dim,),
            dtype=np.float32
        )

    def reset(self):
        self.current_step = 0
        return self._get_obs()

    def _get_obs(self):
        stock_returns = self.stock_returns[self.current_step]
        regime = self.regime_features[self.current_step]

        obs = np.concatenate([stock_returns, regime])
        return np.nan_to_num(obs)

    def step(self, action):
        action_sum = action.sum()
        action = action / action_sum if action_sum != 0 else np.ones_like(action) / len(action)

        # Reward calculated explicitly using raw returns
        reward = np.dot(action, self.raw_stock_returns[self.current_step + 1])

        self.current_step += 1
        done = self.current_step >= len(self.stock_returns) - 1
        obs = self._get_obs() if not done else np.zeros(self.observation_space.shape)
        reward = 0.0 if np.isnan(reward) else reward
        return obs, reward, done, {}


In [11]:
# reb every 21 days
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
import pandas as pd
import numpy as np

model_save_dir = 'E:/XAI/RL_with_SHAP/pythonProject1/SVERL_icml_2023/portfolio_DRL/trained_single_ppo/'

train_window, test_window = 252, 63
rebalance_freq = 21  # rebalance every 21 days
tickers = stock_returns_norm.columns.tolist()
selected_columns = daily_regimes.columns.tolist()
factor_columns = factor_returns_norm.columns.tolist()
use_factor_returns = False
records = []
raw_stock_returns = stock_returns_norm_filter
load_existing_model = False

total_windows = len(range(0, len(merged_data) - train_window - test_window, test_window))

for i, start in enumerate(range(0, len(merged_data) - train_window - test_window, test_window)):
    print(f"\n{'='*30}\nStarting Training Window {i+1}/{total_windows}")
    print(f"Training Period: {merged_data.index[start].date()} to {merged_data.index[start + train_window - 1].date()}")

    # Explicit training and testing dataset slicing
    train_df = merged_data.iloc[start:start + train_window]
    test_df = merged_data.iloc[start + train_window:start + train_window + test_window + 1]
    raw_returns_train = raw_stock_returns.loc[train_df.index]
    raw_returns_test = raw_stock_returns.loc[test_df.index]

    if train_df.isna().values.any() or test_df.isna().values.any():
        print(f"Skipping window {i+1} due to NaNs.")
        continue

    # Train PPO model explicitly
    model_filename = os.path.join(model_save_dir, f"ppo_model_window_{i}.zip")

    # Setup environment
    env = DummyVecEnv([
        lambda: SinglePPOEnv(
            train_df[tickers],
            train_df[selected_columns + factor_columns] if use_factor_returns else train_df[selected_columns],
            raw_returns_train
        )
    ])


     # Load existing model if indicated and file exists
    if load_existing_model and i > 0 and os.path.exists(os.path.join(model_save_dir, f"ppo_model_window_{i-1}.zip")):
        previous_model_filename = os.path.join(model_save_dir, f"ppo_model_window_{i-1}.zip")
        print(f"Loading and retraining previous PPO model from {previous_model_filename}")
        model = PPO.load(previous_model_filename, env=env)
        model.learn(total_timesteps=10000)
    else:
        print("Training new PPO model from scratch...")
        model = PPO('MlpPolicy', env, verbose=0)
        model.learn(total_timesteps=10000)

    model.save(model_filename)
    print(f"Model saved to {model_filename}")

    # Testing phase setup
    test_dates = test_df.index
    test_returns = test_df[tickers].values
    test_regimes = test_df[selected_columns + factor_columns].values if use_factor_returns else test_df[selected_columns].values

    print(f"\nTesting Period: {test_dates[0].date()} to {test_dates[-1].date()} (63 days total)")

    # Evaluate every 21 days (3 rebalances per testing window)
    for rebalance_num, rebalance_start in enumerate(range(0, test_window, rebalance_freq)):
        rebalance_end = min(rebalance_start + rebalance_freq, test_window)

        print(f"\n--- Rebalance {rebalance_num + 1} ---")
        print(f"Rebalance date: {test_dates[rebalance_start].date()}")
        print(f"Holding period: {test_dates[rebalance_start].date()} to {test_dates[rebalance_end - 1].date()}")

        obs_returns = test_returns[rebalance_start:rebalance_end]
        obs_regimes = test_regimes[rebalance_start:rebalance_end]
        obs_raw_returns = raw_returns_test.iloc[rebalance_start:rebalance_end]

        env_test = SinglePPOEnv(
            returns_df=pd.DataFrame(obs_returns, columns=tickers),
            regime_df=pd.DataFrame(obs_regimes, columns=(selected_columns + factor_columns) if use_factor_returns else selected_columns),
            raw_returns_df=obs_raw_returns
        )

        obs = env_test.reset()
        action, _ = model.predict(obs, deterministic=True)

        # Explicit normalization right here:
        action_sum = action.sum()
        if action_sum != 0:
            action /= action_sum
        else:
            action = np.ones_like(action) / len(action)

        weights_dict = {ticker: round(weight, 4) for ticker, weight in zip(tickers, action)}
        print(f"Normalized Portfolio weights:\n{weights_dict}")

        portfolio_value = 1.0
        for step in range(rebalance_end - rebalance_start - 1):
            reward = np.dot(action, obs_raw_returns.iloc[step + 1])
            portfolio_value *= (1 + reward)
        cumulative_reward = portfolio_value - 1.0

        records.append({
            'Rebalance_Date': test_dates[rebalance_start],
            'Window': i,
            'Rebalance_Number': rebalance_num,
            'Holding_Period_Start': test_dates[rebalance_start],
            'Holding_Period_End': test_dates[rebalance_end - 1],
            'Cumulative_Reward': cumulative_reward,
            **weights_dict
        })

        print(f"Cumulative reward for holding period: {cumulative_reward:.4f}")

print("\n" + "="*30 + "\nAll training/testing windows completed successfully.\n" + "="*30)

# Save final results explicitly
result_df = pd.DataFrame(records)
result_df['Cumulative_Return'] = result_df['Cumulative_Reward'].cumsum()
result_df.to_csv('single_step_allocations_dates_rebalanced.csv', index=False)

print("Final results saved to 'single_step_allocations_dates_rebalanced.csv'")



Starting Training Window 1/69
Training Period: 2006-06-09 to 2007-06-11
Training new PPO model from scratch...


E:\XAI\RL_with_SHAP\pythonProject1\.venv\lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


Model saved to E:/XAI/RL_with_SHAP/pythonProject1/SVERL_icml_2023/portfolio_DRL/trained_single_ppo/ppo_model_window_0.zip

Testing Period: 2007-06-12 to 2007-09-11 (63 days total)

--- Rebalance 1 ---
Rebalance date: 2007-06-12
Holding period: 2007-06-12 to 2007-07-11
Normalized Portfolio weights:
{'BA': np.float32(0.1129), 'AMGN': np.float32(0.0), 'DIS': np.float32(0.0149), 'NKE': np.float32(0.0161), 'HON': np.float32(0.0), 'MMM': np.float32(0.0021), 'CAT': np.float32(0.0541), 'KO': np.float32(0.018), 'PG': np.float32(0.0), 'AXP': np.float32(0.0709), 'GS': np.float32(0.0581), 'JPM': np.float32(0.1283), 'MCD': np.float32(0.0), 'HD': np.float32(0.0), 'AAPL': np.float32(0.0167), 'CRM': np.float32(0.0), 'CSCO': np.float32(0.0395), 'IBM': np.float32(0.05), 'MSFT': np.float32(0.0222), 'TRV': np.float32(0.0005), 'UNH': np.float32(0.0379), 'CVX': np.float32(0.0656), 'JNJ': np.float32(0.0146), 'MRK': np.float32(0.0759), 'AMZN': np.float32(0.0601), 'WMT': np.float32(0.0), 'INTC': np.float32(0.0

E:\XAI\RL_with_SHAP\pythonProject1\.venv\lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


Model saved to E:/XAI/RL_with_SHAP/pythonProject1/SVERL_icml_2023/portfolio_DRL/trained_single_ppo/ppo_model_window_1.zip

Testing Period: 2007-09-11 to 2007-12-10 (63 days total)

--- Rebalance 1 ---
Rebalance date: 2007-09-11
Holding period: 2007-09-11 to 2007-10-09
Normalized Portfolio weights:
{'BA': np.float32(0.0), 'AMGN': np.float32(0.0), 'DIS': np.float32(0.0), 'NKE': np.float32(0.0457), 'HON': np.float32(0.0161), 'MMM': np.float32(0.0), 'CAT': np.float32(0.0017), 'KO': np.float32(0.1126), 'PG': np.float32(0.0678), 'AXP': np.float32(0.0), 'GS': np.float32(0.0658), 'JPM': np.float32(0.1007), 'MCD': np.float32(0.015), 'HD': np.float32(0.0), 'AAPL': np.float32(0.0068), 'CRM': np.float32(0.0644), 'CSCO': np.float32(0.0222), 'IBM': np.float32(0.0201), 'MSFT': np.float32(0.0), 'TRV': np.float32(0.0926), 'UNH': np.float32(0.1426), 'CVX': np.float32(0.0065), 'JNJ': np.float32(0.0503), 'MRK': np.float32(0.0052), 'AMZN': np.float32(0.0856), 'WMT': np.float32(0.0), 'INTC': np.float32(0.01

...

Model saved to E:/XAI/RL_with_SHAP/pythonProject1/SVERL_icml_2023/portfolio_DRL/trained_single_ppo/ppo_model_window_66.zip

Testing Period: 2023-12-15 to 2024-03-19 (63 days total)

--- Rebalance 1 ---
Rebalance date: 2023-12-15
Holding period: 2023-12-15 to 2024-01-17
Normalized Portfolio weights:
{'BA': np.float32(0.0), 'AMGN': np.float32(0.0605), 'DIS': np.float32(0.0), 'NKE': np.float32(0.0), 'HON': np.float32(0.0), 'MMM': np.float32(0.0), 'CAT': np.float32(0.0), 'KO': np.float32(0.064), 'PG': np.float32(0.0401), 'AXP': np.float32(0.0641), 'GS': np.float32(0.0268), 'JPM': np.float32(0.0512), 'MCD': np.float32(0.0), 'HD': np.float32(0.0), 'AAPL': np.float32(0.0), 'CRM': np.float32(0.1761), 'CSCO': np.float32(0.0099), 'IBM': np.float32(0.0405), 'MSFT': np.float32(0.0926), 'TRV': np.float32(0.0633), 'UNH': np.float32(0.0), 'CVX': np.float32(0.0), 'JNJ': np.float32(0.0), 'MRK': np.float32(0.0), 'AMZN': np.float32(0.0598), 'WMT': np.float32(0.0862), 'INTC': np.float32(0.1254), 'VZ': np.

E:\XAI\RL_with_SHAP\pythonProject1\.venv\lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


Model saved to E:/XAI/RL_with_SHAP/pythonProject1/SVERL_icml_2023/portfolio_DRL/trained_single_ppo/ppo_model_window_67.zip

Testing Period: 2024-03-19 to 2024-06-18 (63 days total)

--- Rebalance 1 ---
Rebalance date: 2024-03-19
Holding period: 2024-03-19 to 2024-04-17
Normalized Portfolio weights:
{'BA': np.float32(0.0), 'AMGN': np.float32(0.1276), 'DIS': np.float32(0.0), 'NKE': np.float32(0.0455), 'HON': np.float32(0.0), 'MMM': np.float32(0.0), 'CAT': np.float32(0.0173), 'KO': np.float32(0.0), 'PG': np.float32(0.0), 'AXP': np.float32(0.0), 'GS': np.float32(0.0), 'JPM': np.float32(0.0), 'MCD': np.float32(0.015), 'HD': np.float32(0.1206), 'AAPL': np.float32(0.2792), 'CRM': np.float32(0.0573), 'CSCO': np.float32(0.0342), 'IBM': np.float32(0.0), 'MSFT': np.float32(0.0208), 'TRV': np.float32(0.0), 'UNH': np.float32(0.0437), 'CVX': np.float32(0.0597), 'JNJ': np.float32(0.0), 'MRK': np.float32(0.0142), 'AMZN': np.float32(0.0), 'WMT': np.float32(0.0302), 'INTC': np.float32(0.1346), 'VZ': np.

E:\XAI\RL_with_SHAP\pythonProject1\.venv\lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


Model saved to E:/XAI/RL_with_SHAP/pythonProject1/SVERL_icml_2023/portfolio_DRL/trained_single_ppo/ppo_model_window_68.zip

Testing Period: 2024-06-18 to 2024-09-18 (63 days total)

--- Rebalance 1 ---
Rebalance date: 2024-06-18
Holding period: 2024-06-18 to 2024-07-18
Normalized Portfolio weights:
{'BA': np.float32(0.0), 'AMGN': np.float32(0.0), 'DIS': np.float32(0.0), 'NKE': np.float32(0.0), 'HON': np.float32(0.0906), 'MMM': np.float32(0.0522), 'CAT': np.float32(0.0118), 'KO': np.float32(0.0), 'PG': np.float32(0.1859), 'AXP': np.float32(0.0), 'GS': np.float32(0.0), 'JPM': np.float32(0.0), 'MCD': np.float32(0.0242), 'HD': np.float32(0.0553), 'AAPL': np.float32(0.106), 'CRM': np.float32(0.0), 'CSCO': np.float32(0.0), 'IBM': np.float32(0.0347), 'MSFT': np.float32(0.0), 'TRV': np.float32(0.0), 'UNH': np.float32(0.0), 'CVX': np.float32(0.0451), 'JNJ': np.float32(0.1679), 'MRK': np.float32(0.015), 'AMZN': np.float32(0.0622), 'WMT': np.float32(0.1012), 'INTC': np.float32(0.0479), 'VZ': np.f

In [ ]:
print("\n" + "="*30 + "\nAll training/testing windows completed successfully.\n" + "="*30)

# Save final results explicitly
result_df = pd.DataFrame(records)
result_df['Cumulative_Return'] = result_df['Cumulative_Reward'].cumsum()
result_df.to_csv('single_step_allocations_dates_rebalanced.csv', index=False)


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, calinski_harabasz_score
# Load data explicitly
shap_metrics = pd.read_csv('shap_value_metrics_export.csv', parse_dates=['Start_Date', 'End_Date'])
factor_returns = pd.read_csv('aligned_factors.csv', parse_dates=['Date'], index_col='Date')
stock_returns = pd.read_csv('daily_returns.csv', parse_dates=['Date'], index_col='Date')

# Explicitly filter only 'Test' phase SHAP metrics
shap_metrics_test = shap_metrics[shap_metrics['Phase'] == 'Test'].copy()

# Verify explicitly
print(f"Total SHAP records (Test Phase): {len(shap_metrics_test)}")

# Select SHAP metrics explicitly
selected_columns = [
    'mean_abs_shap_Mkt-RF', 'mean_abs_shap_SMB', 'mean_abs_shap_HML',
    'mean_abs_shap_RMW', 'mean_abs_shap_CMA',
    'shap_std_Mkt-RF', 'shap_std_SMB', 'shap_std_HML',
    'shap_std_RMW', 'shap_std_CMA',
    'mean_abs_over_std_Mkt-RF', 'mean_abs_over_std_SMB', 'mean_abs_over_std_HML',
    'mean_abs_over_std_RMW', 'mean_abs_over_std_CMA'
]

# Aggregate SHAP metrics explicitly by Start_Date and End_Date across stocks
shap_agg = shap_metrics_test.groupby(['Start_Date', 'End_Date'])[selected_columns].mean().reset_index()

# Explicitly scale SHAP metrics
scaler_shap = StandardScaler()
shap_agg[selected_columns] = scaler_shap.fit_transform(shap_agg[selected_columns])

# Verify explicitly scaled SHAP metrics
# print("Scaled SHAP metrics (Test phase):", shap_agg.head())

# Explicitly map SHAP intervals to daily returns
daily_regimes = pd.DataFrame(index=stock_returns.index)

for _, row in shap_agg.iterrows():
    mask = (daily_regimes.index >= row['Start_Date']) & (daily_regimes.index <= row['End_Date'])
    daily_regimes.loc[mask, selected_columns] = row[selected_columns].values

# Explicit forward-fill for continuity
daily_regimes.ffill(inplace=True)
daily_regimes.dropna(inplace=True)

# Perform KMeans clustering explicitly on daily regimes
kmeans = KMeans(n_clusters=2, random_state=42, n_init=10)
daily_regimes['Cluster'] = kmeans.fit_predict(daily_regimes[selected_columns])
# daily_regimes['Cluster'] = 1 - daily_regimes['Cluster']
# cluster_mapping = {0: 1, 1: 0}  # Good explicitly set as 1
# daily_regimes['Cluster_Mapped'] = daily_regimes['Cluster'].map(cluster_mapping)

from sklearn.cluster import KMeans

# Example: Clustering raw returns into regimes
# kmeans = KMeans(n_clusters=2, random_state=42, n_init=10)
# regime_labels = kmeans.fit_predict(raw_future_returns_df)
# regime_labels_series = pd.Series(regime_labels, index=raw_future_returns_df.index)

# Verify explicitly aligned daily regimes
# print("Daily regimes (aligned):", daily_regimes.head())

# Normalize explicitly factor returns
factor_scaler = StandardScaler()
factor_returns_norm = pd.DataFrame(
    factor_scaler.fit_transform(factor_returns),
    index=factor_returns.index,
    columns=factor_returns.columns
)

# Normalize explicitly stock returns
stock_scaler = StandardScaler()
stock_returns_norm = pd.DataFrame(
    stock_scaler.fit_transform(stock_returns),
    index=stock_returns.index,
    columns=stock_returns.columns
)

# Merge explicitly into final dataset
merged_data = stock_returns_norm.join(daily_regimes[['Cluster'] + selected_columns], how='inner')\
                                .join(factor_returns_norm, rsuffix='_factor')\
                                .dropna()

# Explicit final dataset verification
# print("Merged normalized data preview:", merged_data.head())

# Save explicitly final aligned dataset
merged_data.to_csv("final_merged_data.csv")
# Explicitly filter raw stock returns to match the merged_data dates
start_date = merged_data.index.min()
end_date = merged_data.index.max()

# Explicitly filter raw stock returns to match the merged_data dates
raw_stock_returns = stock_returns.loc[start_date:end_date].copy()
raw_stock_returns.to_csv("aligned_raw_stock_returns.csv")

stock_returns_norm_filter = stock_returns_norm.loc[start_date:end_date].copy()
cluster_avg_returns = raw_stock_returns.join(daily_regimes['Cluster']).groupby('Cluster').mean().mean(axis=1)
print("Explicit Cluster Mean Returns Check:\n", cluster_avg_returns)


In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, calinski_harabasz_score
import matplotlib.pyplot as plt

# Use existing explicitly scaled SHAP metrics from daily_regimes
regime_data = daily_regimes.values
cluster_range = range(2, 11)  # Evaluate 2 to 10 clusters explicitly
inertia = []
silhouette_scores = []
calinski_scores = []

for k in cluster_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = kmeans.fit_predict(regime_data)

    inertia.append(kmeans.inertia_)
    silhouette_scores.append(silhouette_score(regime_data, labels))
    calinski_scores.append(calinski_harabasz_score(regime_data, labels))

    print(f"Clusters: {k} | Inertia: {kmeans.inertia_:.2f} | "
          f"Silhouette: {silhouette_scores[-1]:.4f} | "
          f"Calinski-Harabasz: {calinski_scores[-1]:.2f}")
fig, ax = plt.subplots(1, 3, figsize=(18, 5))

ax[0].plot(cluster_range, inertia, 'bo-', linewidth=2)
ax[0].set_title('Elbow Method (Inertia)', fontsize=14)
ax[0].set_xlabel('Number of Clusters', fontsize=12)
ax[0].set_ylabel('Inertia', fontsize=12)

ax[1].plot(cluster_range, silhouette_scores, 'go-', linewidth=2)
ax[1].set_title('Silhouette Score', fontsize=14)
ax[1].set_xlabel('Number of Clusters', fontsize=12)
ax[1].set_ylabel('Silhouette Score', fontsize=12)

ax[2].plot(cluster_range, calinski_scores, 'ro-', linewidth=2)
ax[2].set_title('Calinski-Harabasz Index', fontsize=14)
ax[2].set_xlabel('Number of Clusters', fontsize=12)
ax[2].set_ylabel('Calinski-Harabasz Score', fontsize=12)

plt.tight_layout()
plt.show()

In [ ]:
import gym
import numpy as np
from gym import spaces

class RegimeSelectorEnv(gym.Env):
    def __init__(self, regime_features_df, raw_future_returns_df):
        super().__init__()
        self.regime_features = regime_features_df.values
        # self.raw_future_returns = raw_future_returns_df.values
        self.raw_future_returns = raw_future_returns_df
        
        self.current_step = 0
        self.n_regimes = 2  # Regimes: 0 = bad, 1 = good

        self.action_space = spaces.Discrete(self.n_regimes)
        self.observation_space = spaces.Box(
            low=-np.inf, high=np.inf, 
            shape=(self.regime_features.shape[1],),
            dtype=np.float32
        )

    def reset(self):
        self.current_step = 0
        return self.regime_features[self.current_step]

    def step(self, action):
        avg_future_return = np.mean(self.raw_future_returns[self.current_step])

        # Reward logic explicitly explained:
        # - action = 1 (predict good regime): reward = avg_future_return
        # - action = 0 (predict bad regime): reward = -avg_future_return
        # Thus, correct predictions yield positive rewards, incorrect yield negative rewards.
        reward = avg_future_return if action == 1 else -avg_future_return

        self.current_step += 1
        done = self.current_step >= len(self.regime_features) - 1

        obs = (
            self.regime_features[self.current_step]
            if not done else np.zeros(self.observation_space.shape)
        )

        return obs, reward, done, {}



import gym
import numpy as np
from gym import spaces

class RegimeConditionedPortfolioEnv(gym.Env):
    def __init__(self, returns_df, chosen_regime_vector, raw_returns_df, factor_returns_df=None):
        super().__init__()
        
        self.stock_returns = returns_df.values
        self.chosen_regime_vector = chosen_regime_vector
        self.raw_stock_returns = raw_returns_df.values
        self.factor_returns = factor_returns_df.values if factor_returns_df is not None else None
        
        self.current_step = 0
        self.n_assets = self.stock_returns.shape[1]

        self.action_space = spaces.Box(low=0, high=1, shape=(self.n_assets,), dtype=np.float32)
        
        obs_dim = self.n_assets + 1
        if self.factor_returns is not None:
            obs_dim += self.factor_returns.shape[1]

        self.observation_space = spaces.Box(
            low=-np.inf, high=np.inf,
            shape=(obs_dim,),
            dtype=np.float32
        )

    def reset(self):
        self.current_step = 0
        return self._get_obs()

    def _get_obs(self):
        returns = self.stock_returns[self.current_step]
        regime = np.array([self.chosen_regime_vector[self.current_step]])
        obs = np.concatenate([returns, regime])

        if self.factor_returns is not None:
            factor_obs = self.factor_returns[self.current_step]
            obs = np.concatenate([obs, factor_obs])

        return np.nan_to_num(obs)

    def step(self, action):
        action_sum = action.sum()
        if action_sum == 0:
            action = np.ones_like(action) / len(action)
        else:
            action = action / action_sum

        # Intuitive reward: explicitly realized portfolio return
        reward = np.dot(action, self.raw_stock_returns[self.current_step + 1])

        self.current_step += 1
        done = self.current_step >= len(self.stock_returns) - 1

        obs = self._get_obs() if not done else np.zeros(self.observation_space.shape)

        reward = 0.0 if np.isnan(reward) else reward

        return obs, reward, done, {}


In [ ]:
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
import os
import pandas as pd
import numpy as np

model_save_dir = 'E:/XAI/RL_with_SHAP/pythonProject1/SVERL_icml_2023/portfolio_DRL/trained_hierarchical_model/'

train_window, test_window = 252, 63
rebalance_freq = 21
use_factor_returns = False  # Set this flag as needed
tickers = stock_returns_norm.columns.tolist()
# regime_columns = daily_regimes.columns.tolist() #### remove cluster in the feature set!!
regime_columns = daily_regimes.columns.drop('Cluster').tolist()
# regime_columns = daily_regimes[['Cluster']].columns.tolist()
factor_columns = factor_returns_norm.columns.tolist() if use_factor_returns else []
records = []
raw_stock_returns = stock_returns_norm_filter
load_existing_model = False
total_windows = len(range(0, len(merged_data) - train_window - test_window, test_window))

for i, start in enumerate(range(0, len(merged_data) - train_window - test_window, test_window)):
    print(f"\n{'='*30}\nStarting Hierarchical Training Window {i+1}/{total_windows}")
    high_model_path = os.path.join(model_save_dir, f"high_model_window_{i}.zip")

    train_df = merged_data.iloc[start:start + train_window]
    test_df = merged_data.iloc[start + train_window:start + train_window + test_window + 1]
    raw_returns_train = raw_stock_returns.loc[train_df.index]
    raw_returns_test = raw_stock_returns.loc[test_df.index]

    if train_df.isna().values.any() or test_df.isna().values.any():
        print(f"Skipping window {i+1} due to NaNs.")
        continue

    # High-Level training data preparation (no look-ahead bias)
    regime_features_high = train_df[regime_columns + factor_columns][:-rebalance_freq]
    future_returns_high = np.array([
        raw_returns_train.iloc[idx + 1: idx + rebalance_freq + 1].mean().mean()
        for idx in range(len(regime_features_high))
    ])

    high_env = DummyVecEnv([
        lambda: RegimeSelectorEnv(regime_features_high, future_returns_high)
    ])
    
    
    # Load existing high-level model or train a new one
    if load_existing_model and os.path.exists(high_model_path):
        print("Loading existing High-Level Regime Selector model...")
        high_model = PPO.load(high_model_path, env=high_env)
        high_model.learn(total_timesteps=3000)  # continue training briefly
        print(f"Continued training on High-Level model. Saved again to {high_model_path}.")
    else:
        print("Training new High-Level Regime Selector model...")
        # policy_kwargs = dict(net_arch=[128, 64], activation_fn=nn.Tanh)
        # 
        # high_model = PPO(
        #     'MlpPolicy', high_env, verbose=0,
        #     learning_rate=1e-4,
        #     n_steps=2048,
        #     batch_size=128,
        #     gamma=0.95,
        #     ent_coef=0.005,  # Slightly lower entropy encourages differentiation
        #     clip_range=0.2,
        #     n_epochs=10,
        #     policy_kwargs=policy_kwargs
        # )
        # high_model.learn(total_timesteps=10000)
    
        print("Training High-Level Regime Selector...")
        high_model = PPO('MlpPolicy', high_env, verbose=0)
        high_model.learn(total_timesteps=50000)
    # high_model_path = os.path.join(model_save_dir, f"high_model_window_{i}.zip")
    # high_model.save(high_model_path)
    # print(f"High-level model saved at: {high_model_path}")

    # Generate historical regimes for low-level model training (no look-ahead)
    historical_regimes = []
    for idx in range(len(regime_features_high)):
        obs = regime_features_high.iloc[idx].values
        regime, _ = high_model.predict(obs, deterministic=True)
        historical_regimes.append(int(regime))

    historical_regimes = pd.Series(historical_regimes, index=regime_features_high.index)

    # Train low-level models per historical regime
    low_level_models = {}
    for regime in historical_regimes.unique():
        low_model_path = os.path.join(model_save_dir, f"low_model_window_{i}_regime_{regime}.zip")
        regime_indices = historical_regimes[historical_regimes == regime].index
        regime_df = train_df.loc[regime_indices]
        chosen_regime_vector_train = np.repeat(regime, len(regime_df))

        low_env_train = DummyVecEnv([
            lambda: RegimeConditionedPortfolioEnv(
                returns_df=regime_df[tickers],
                chosen_regime_vector=chosen_regime_vector_train,
                raw_returns_df=raw_returns_train.loc[regime_df.index]
            )
        ])
        
        if load_existing_model and os.path.exists(low_model_path):
            print(f"Loading existing Low-Level model for regime {regime}...")
            low_model = PPO.load(low_model_path, env=low_env_train)
            low_model.learn(total_timesteps=3000)  # brief continued training
            print(f"Continued training on Low-Level model regime {regime}. Saved again to {low_model_path}.")
        else:
            print(f"Training new Low-Level model for regime {regime}...")
            # low_model = PPO(
            #     'MlpPolicy', low_env_train, verbose=0,
            #     learning_rate=5e-5,
            #     n_steps=2048,
            #     batch_size=128,
            #     gamma=0.97,
            #     ent_coef=0.003,  # Reduced entropy to discourage uniform policy
            #     clip_range=0.2,
            #     n_epochs=10,
            #     policy_kwargs=policy_kwargs
            # )
            # low_model.learn(total_timesteps=10000)

            low_model = PPO('MlpPolicy', low_env_train, verbose=0)
            low_model.learn(total_timesteps=50000)
        # low_model_path = os.path.join(model_save_dir, f"low_model_window_{i}_regime_{regime}.zip")
        # low_model.save(low_model_path)
        low_level_models[regime] = low_model
        # print(f"Low-level model for regime {regime} saved at: {low_model_path}")

    # Testing
    test_dates = test_df.index
    test_returns = test_df[tickers].values
    test_regimes = test_df[regime_columns + factor_columns].values

    for rebalance_num, rebalance_start in enumerate(range(0, test_window, rebalance_freq)):
        rebalance_end = min(rebalance_start + rebalance_freq, test_window)

        high_obs = test_regimes[rebalance_start]
        chosen_regime, _ = high_model.predict(high_obs, deterministic=True)
        chosen_regime = int(chosen_regime.item())
        # high_predictions = [high_model.predict(high_obs, deterministic=False)[0].item() for _ in range(200)]
        # chosen_regime = int(pd.Series(high_predictions).mode()[0])  # take most common predicted regime
        
        chosen_regime_vector_test = np.repeat(chosen_regime, rebalance_end - rebalance_start)
        print(f'chosen_regime = {chosen_regime} and check is {chosen_regime in low_level_models}')
        if chosen_regime in low_level_models:
            low_model = low_level_models[chosen_regime]
            low_env_test = RegimeConditionedPortfolioEnv(
                returns_df=pd.DataFrame(test_returns[rebalance_start:rebalance_end], columns=tickers),
                chosen_regime_vector=chosen_regime_vector_test,
                raw_returns_df=raw_returns_test.iloc[rebalance_start:rebalance_end]
            )

            low_obs = low_env_test.reset()
            action, _ = low_model.predict(low_obs, deterministic=True)
            action /= action.sum()
            # action_samples = np.array([
            #     low_model.predict(low_obs, deterministic=False)[0]
            #     for _ in range(200)
            # ])
            # action_mean = action_samples.mean(axis=0)
            # action_mean /= action_mean.sum()  # normalization
            # action = action_mean
            
        else:
            action = np.ones(len(tickers)) / len(tickers)

        portfolio_value = 1.0
        for step in range(rebalance_end - rebalance_start - 1):
            reward = np.dot(action, raw_returns_test.iloc[rebalance_start + step + 1])
            portfolio_value *= (1 + reward)
        cumulative_reward = portfolio_value - 1.0

        weights_dict = {ticker: round(weight, 4) for ticker, weight in zip(tickers, action)}
        records.append({
            'Rebalance_Date': test_dates[rebalance_start],
            'Window': i,
            'Rebalance_Number': rebalance_num,
            'Chosen_Regime': chosen_regime,
            'Holding_Period_Start': test_dates[rebalance_start],
            'Holding_Period_End': test_dates[rebalance_end - 1],
            'Cumulative_Reward': cumulative_reward,
            **weights_dict
        })

        print(f"Cumulative reward: {cumulative_reward:.4f}")

result_df = pd.DataFrame(records)
result_df['Cumulative_Return'] = result_df['Cumulative_Reward'].cumsum()
result_df.to_csv('hierarchical_allocations_dates_rebalanced.csv', index=False)

print("\nHierarchical training/testing completed successfully.")
print("Final hierarchical results saved to 'hierarchical_allocations_dates_rebalanced.csv'")


In [4]:
############################
# retrain model using the previous trained agents for high and low:
############################

from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
import os
import pandas as pd
import numpy as np

model_save_dir = 'E:/XAI/RL_with_SHAP/pythonProject1/SVERL_icml_2023/portfolio_DRL/trained_hierarchical_model/'

train_window, test_window = 252, 63
rebalance_freq = 21
use_factor_returns = True  # Set this flag as needed
tickers = stock_returns_norm.columns.tolist()
# regime_columns = daily_regimes.columns.tolist() 
# regime_columns = daily_regimes.columns.drop('Cluster').tolist() #### remove cluster in the feature set!!
regime_columns = daily_regimes[['Cluster']].columns.tolist()
factor_columns = factor_returns_norm.columns.tolist() if use_factor_returns else []
records = []
raw_stock_returns = stock_returns_norm_filter
load_existing_model = True
total_windows = len(range(0, len(merged_data) - train_window - test_window, test_window))

for i, start in enumerate(range(0, len(merged_data) - train_window - test_window, test_window)):
    print(f"\n{'='*30}\nStarting Hierarchical Training Window {i+1}/{total_windows}")
    
    # Model paths explicitly defined for high and low-level models
    high_model_path = os.path.join(model_save_dir, f"high_model_window_{i-1}.zip")  # Load previous model
    new_high_model_path = os.path.join(model_save_dir, f"high_model_window_{i}.zip")  # Save current model
    
    # Data splitting
    train_df = merged_data.iloc[start:start + train_window]
    test_df = merged_data.iloc[start + train_window:start + train_window + test_window + 1]
    raw_returns_train = raw_stock_returns.loc[train_df.index]
    raw_returns_test = raw_stock_returns.loc[test_df.index]

    # High-Level training data preparation (no look-ahead bias)
    regime_features_high = train_df[regime_columns + factor_columns][:-rebalance_freq]
    future_returns_high = np.array([
        raw_returns_train.iloc[idx + 1: idx + rebalance_freq + 1].mean().mean()
        for idx in range(len(regime_features_high))
    ])

    high_env = DummyVecEnv([
        lambda: RegimeSelectorEnv(regime_features_high, future_returns_high)
    ])
    
    # Load and continue training from previous window's model
    if load_existing_model and i > 0 and os.path.exists(high_model_path):
        print("Loading existing High-Level model from previous window...")
        high_model = PPO.load(high_model_path, env=high_env)
        high_model.learn(total_timesteps=20000)  # shorter fine-tuning
    else:
        print("Training new High-Level Regime Selector from scratch...")
        high_model = PPO('MlpPolicy', high_env, verbose=0)
        high_model.learn(total_timesteps=50000)  # full training only for first window
    
    # Save updated high-level model explicitly for next window's reuse
    high_model.save(new_high_model_path)
    print(f"High-level model saved at: {new_high_model_path}")

    # Generate historical regimes using the trained high-level model
    historical_regimes = []
    for idx in range(len(regime_features_high)):
        obs = regime_features_high.iloc[idx].values
        regime, _ = high_model.predict(obs, deterministic=True)
        historical_regimes.append(int(regime))

    historical_regimes = pd.Series(historical_regimes, index=regime_features_high.index)

    # Low-level model training with incremental training
    low_level_models = {}
    for regime in historical_regimes.unique():
        prev_low_model_path = os.path.join(model_save_dir, f"low_model_window_{i-1}_regime_{regime}.zip")
        current_low_model_path = os.path.join(model_save_dir, f"low_model_window_{i}_regime_{regime}.zip")
        
        regime_indices = historical_regimes[historical_regimes == regime].index
        regime_df = train_df.loc[regime_indices]
        chosen_regime_vector_train = np.repeat(regime, len(regime_df))

        low_env_train = DummyVecEnv([
            lambda: RegimeConditionedPortfolioEnv(
                returns_df=regime_df[tickers],
                chosen_regime_vector=chosen_regime_vector_train,
                raw_returns_df=raw_returns_train.loc[regime_df.index]
            )
        ])
        
        if load_existing_model and i > 0 and os.path.exists(prev_low_model_path):
            print(f"Loading existing Low-Level model for regime {regime} from previous window...")
            low_model = PPO.load(prev_low_model_path, env=low_env_train)
            low_model.learn(total_timesteps=20000)  # shorter fine-tuning
        else:
            print(f"Training new Low-Level model for regime {regime} from scratch...")
            low_model = PPO('MlpPolicy', low_env_train, verbose=0)
            low_model.learn(total_timesteps=50000)  # full training for first window or new regime

        low_model.save(current_low_model_path)
        low_level_models[regime] = low_model
        print(f"Low-level model for regime {regime} saved at: {current_low_model_path}")


    # Testing
    test_dates = test_df.index
    test_returns = test_df[tickers].values
    test_regimes = test_df[regime_columns + factor_columns].values

    for rebalance_num, rebalance_start in enumerate(range(0, test_window, rebalance_freq)):
        rebalance_end = min(rebalance_start + rebalance_freq, test_window)

        high_obs = test_regimes[rebalance_start]
        chosen_regime, _ = high_model.predict(high_obs, deterministic=True)
        chosen_regime = int(chosen_regime.item())
        # high_predictions = [high_model.predict(high_obs, deterministic=False)[0].item() for _ in range(200)]
        # chosen_regime = int(pd.Series(high_predictions).mode()[0])  # take most common predicted regime
        
        chosen_regime_vector_test = np.repeat(chosen_regime, rebalance_end - rebalance_start)
        print(f'chosen_regime = {chosen_regime} and check is {chosen_regime in low_level_models}')
        if chosen_regime in low_level_models:
            low_model = low_level_models[chosen_regime]
            low_env_test = RegimeConditionedPortfolioEnv(
                returns_df=pd.DataFrame(test_returns[rebalance_start:rebalance_end], columns=tickers),
                chosen_regime_vector=chosen_regime_vector_test,
                raw_returns_df=raw_returns_test.iloc[rebalance_start:rebalance_end]
            )

            low_obs = low_env_test.reset()
            action, _ = low_model.predict(low_obs, deterministic=True)
            action /= action.sum()
            # action_samples = np.array([
            #     low_model.predict(low_obs, deterministic=False)[0]
            #     for _ in range(200)
            # ])
            # action_mean = action_samples.mean(axis=0)
            # action_mean /= action_mean.sum()  # normalization
            # action = action_mean
            
        else:
            action = np.ones(len(tickers)) / len(tickers)

        portfolio_value = 1.0
        for step in range(rebalance_end - rebalance_start - 1):
            reward = np.dot(action, raw_returns_test.iloc[rebalance_start + step + 1])
            portfolio_value *= (1 + reward)
        cumulative_reward = portfolio_value - 1.0

        weights_dict = {ticker: round(weight, 4) for ticker, weight in zip(tickers, action)}
        records.append({
            'Rebalance_Date': test_dates[rebalance_start],
            'Window': i,
            'Rebalance_Number': rebalance_num,
            'Chosen_Regime': chosen_regime,
            'Holding_Period_Start': test_dates[rebalance_start],
            'Holding_Period_End': test_dates[rebalance_end - 1],
            'Cumulative_Reward': cumulative_reward,
            **weights_dict
        })

        print(f"Cumulative reward: {cumulative_reward:.4f}")

result_df = pd.DataFrame(records)
result_df['Cumulative_Return'] = result_df['Cumulative_Reward'].cumsum()
result_df.to_csv('hierarchical_allocations_dates_rebalanced.csv', index=False)

print("\nHierarchical training/testing completed successfully.")
print("Final hierarchical results saved to 'hierarchical_allocations_dates_rebalanced.csv'")




Starting Hierarchical Training Window 1/69
Training new High-Level Regime Selector from scratch...


E:\XAI\RL_with_SHAP\pythonProject1\.venv\lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


High-level model saved at: E:/XAI/RL_with_SHAP/pythonProject1/SVERL_icml_2023/portfolio_DRL/trained_hierarchical_model/high_model_window_0.zip
Training new Low-Level model for regime 1 from scratch...


E:\XAI\RL_with_SHAP\pythonProject1\.venv\lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


Low-level model for regime 1 saved at: E:/XAI/RL_with_SHAP/pythonProject1/SVERL_icml_2023/portfolio_DRL/trained_hierarchical_model/low_model_window_0_regime_1.zip
Training new Low-Level model for regime 0 from scratch...


...

High-level model saved at: E:/XAI/RL_with_SHAP/pythonProject1/SVERL_icml_2023/portfolio_DRL/trained_hierarchical_model/high_model_window_68.zip
Loading existing Low-Level model for regime 0 from previous window...


E:\XAI\RL_with_SHAP\pythonProject1\.venv\lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


Low-level model for regime 0 saved at: E:/XAI/RL_with_SHAP/pythonProject1/SVERL_icml_2023/portfolio_DRL/trained_hierarchical_model/low_model_window_68_regime_0.zip
Loading existing Low-Level model for regime 1 from previous window...


E:\XAI\RL_with_SHAP\pythonProject1\.venv\lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


Low-level model for regime 1 saved at: E:/XAI/RL_with_SHAP/pythonProject1/SVERL_icml_2023/portfolio_DRL/trained_hierarchical_model/low_model_window_68_regime_1.zip
chosen_regime = 0 and check is True
Cumulative reward: 0.1713
chosen_regime = 1 and check is True
Cumulative reward: -0.7580
chosen_regime = 0 and check is True
Cumulative reward: -0.3835

Hierarchical training/testing completed successfully.
Final hierarchical results saved to 'hierarchical_allocations_dates_rebalanced.csv'


In [ ]:
result_df = pd.DataFrame(records)
result_df['Cumulative_Return'] = result_df['Cumulative_Reward'].cumsum()
result_df.to_csv('hierarchical_allocations_dates_rebalanced.csv', index=False)

print("\nHierarchical training/testing completed successfully.")
print("Final hierarchical results saved to 'hierarchical_allocations_dates_rebalanced.csv'")


In [ ]:
import pandas as pd
import numpy as np

# Load data explicitly
port_wts = pd.read_csv('port_wt_test.csv', parse_dates=['Date'], index_col='Date')
daily_returns = pd.read_csv('daily_returns.csv', parse_dates=['Date'], index_col='Date')

common_tickers = [col for col in port_wts.columns if col in daily_returns.columns]
daily_returns = daily_returns[common_tickers]

# Explicitly filter daily returns to the date range covered by portfolio weights
start_date, end_date = port_wts.index.min(), port_wts.index.max() + pd.Timedelta(days=30)
daily_returns = daily_returns.loc[start_date:end_date]

# Initialize drifted weights with the first available rebalance weights
initial_weights = port_wts.loc[start_date].values

equal_weight = np.array([1.0 / len(common_tickers)] * len(common_tickers))

# Create drifted weights DataFrame explicitly initialized
drifted_weights = pd.DataFrame(index=daily_returns.index, columns=common_tickers)
equal_weights = pd.DataFrame(index=daily_returns.index, columns=common_tickers)

current_weights = initial_weights
current_equal_weights = equal_weight

# Initialize returns DataFrame explicitly
returns_df = pd.DataFrame(index=daily_returns.index, columns=['Optimal_Portfolio_Return', 'Equal_Weight_Return'])

for current_date in daily_returns.index:
    if current_date in port_wts.index:
        # Explicit rebalance date: assign new weights
        current_weights = port_wts.loc[current_date].values
        current_equal_weights = equal_weight
    else:
        # Explicitly drift weights using previous day's return
        prev_day_return = daily_returns.loc[current_date]

        drifted_wts_numerator = current_weights * (1 + prev_day_return.values)
        current_weights = drifted_wts_numerator / np.sum(drifted_wts_numerator)

        equal_drifted_numerator = current_equal_weights * (1 + prev_day_return.values)
        current_equal_weights = equal_drifted_numerator / np.sum(equal_drifted_numerator)

    drifted_weights.loc[current_date] = current_weights
    equal_weights.loc[current_date] = current_equal_weights
    shifted_drifted_weights = drifted_weights.shift(1)
    shifted_equal_weights = equal_weights.shift(1)
    if current_date == daily_returns.index[0]:
        # On the first day, use initial weights directly
        returns_df.loc[current_date, 'Optimal_Portfolio_Return'] = np.dot(
            drifted_weights.loc[current_date], daily_returns.loc[current_date])
        returns_df.loc[current_date, 'Equal_Weight_Return'] = np.dot(
            equal_weights.loc[current_date], daily_returns.loc[current_date])
    else:
        # Explicitly use previous day's weights
        returns_df.loc[current_date, 'Optimal_Portfolio_Return'] = np.dot(
            shifted_drifted_weights.loc[current_date], daily_returns.loc[current_date])
        returns_df.loc[current_date, 'Equal_Weight_Return'] = np.dot(
            shifted_equal_weights.loc[current_date], daily_returns.loc[current_date])

# Check explicitly
print("Drifted weights (head):\n", drifted_weights.head())
print("\nPortfolio returns (head):\n", returns_df.head())

# Save explicitly
drifted_weights.to_csv('drifted_weights_corrected.csv')
equal_weights.to_csv('equal_weights.csv')
returns_df.to_csv('portfolio_returns_combined.csv')

